# Запуск Экспериментов в Azure ML

## 1. Соединение со своим Workspace

Импорт модулей:

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.datastore import Datastore

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import json


# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Создаем соединение со своим Workspace:

In [ ]:
ws = Workspace.from_config()
print('Connected Workspace:', ws.name, ws.resource_group, ws.location, sep = '\n')

## 2. Загрузка данных для Эксперимента

Воспользуемся данными загружеными на [прошлом лабораторном занятии](../module-3/azure-datastore.ipynb):

In [ ]:
# Get uploaded data 
ds = ws.get_default_datastore()
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))

data = diabetes_ds.to_pandas_dataframe()
data[1:10]

## 3. Запуск Эксперимента

Создадим Эксперимент и залоггируем основыне данные об эксперименте и данных используемых в нем:

In [ ]:
# Create an Azure ML experiment
experiment = Experiment(workspace=ws, name='diabetes-experiment')

# Create an interactive run
run = experiment.start_logging()
run.tag('diabetes')
run.tag('azure-ml-course')

run.log('Dataset dimension', f'{data.shape[0]}x{data.shape[1]}')
run.log('Yet another important fact', 'Think about it')

print(f'Starting experiment {experiment.name}...')

Проанализируем количество пациентов с диабетом и без и залогируем результаты:

In [ ]:
# Plot and log the count of diabetic vs non-diabetic patients
diabetic_counts = data['Diabetic'].value_counts()

fig = plt.figure(figsize=(6,6))
ax = fig.gca()    

diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name = 'label distribution', plot = fig)

# log distinct pregnancy counts
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

Посчитаем описательные статистики датасета и залогируем результаты:

In [ ]:
# Log summary statistics for numeric columns
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat = keys[index], value = values[index])

Сохраним 100 случайных строк из дасета в выходные данные Эксперимента:

In [ ]:
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream = './sample.csv')

Завершим эксперимент:

In [ ]:
# Complete the run
run.complete()

## 4. Просмотр результатов Эксперимента

Детали эксперимента:

In [ ]:
details = run.get_details()
print(details)

Метрики эксперимента:

In [ ]:
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

Выходные данные:

In [ ]:
files = run.get_file_names()
print(json.dumps(files, indent=2))

Визуализируем данные с помощью виджета:

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

Перейдите по сссылке, которая появилась в виджете, чтобы просмотеть результаты Эксперимента на протале Azure ML.